<a href="https://colab.research.google.com/github/vadaliah/CS5260/blob/master/VWAP%20ML%20Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Forex VWAP(Volume Weighted Average Price) ML Solution using scikit-learn**

This notebook demonstrates Machine Learning solution to predict VWAP direction for given currency pair based on historical volume dataset

**Problem Formulation**

In this example, we will use Historical Currencypair price volume dataset provided from FOREX Tester APP, available here: https://forextester.com/data/datasources.

The dataset contains Hourly pricing data of 6 currency pair (EURUSD, GBPUSD, AUDUSD, NZSUSD, USDJPY and USDCHF) for April 2018.

EDA Validation:
1. Ensure data is complete, verify presence of  data for all 6 currency pair is present in the volume dataset
2.  Verify count of data by currencypair, businessdate combination.
  Any currencypair with insufficient data will skew VWAP results and result in inaccurate model outcome
3. Verify Trading dataset is complete, contains currency pair and business dates in scope
4. Esnure Join between CurrencyPair volume and Trading dataset is successful and there is matching attributes in both the datasets
